<a href="https://colab.research.google.com/github/sammyamajumdar/ELEC460/blob/main/ELEC460_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib import colors
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow import keras
import keras.backend as K

In [4]:
data = pd.read_excel("/content/regularised_data (1).xlsx")
data = data.drop(columns=["Unnamed: 0"])
target_vector = data["target"]
data = data.drop(columns=["target"])
X = data
X_train, X_test, y_train, y_test = train_test_split(X, target_vector, test_size=0.1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1) # 0.25 x 0.8 = 0.2
print("Train set size: ", X_train.shape)
print("Train labels size: ", y_train.shape)

Train set size:  (4470, 62)
Train labels size:  (4470,)


In [5]:
model = keras.models.Sequential([
    keras.layers.Dense(32, input_dim=62, kernel_initializer="normal",activation="relu"),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1)
])
model.compile(loss="mean_squared_error", optimizer="Adam", metrics=["RootMeanSquaredError"])
my_model = model.fit(X_train,y_train,epochs=300,verbose=2,validation_split=0.2,shuffle=True)

Epoch 1/300
112/112 - 1s - loss: 1943.2861 - root_mean_squared_error: 44.0827 - val_loss: 553.7179 - val_root_mean_squared_error: 23.5312
Epoch 2/300
112/112 - 0s - loss: 500.9340 - root_mean_squared_error: 22.3816 - val_loss: 446.9974 - val_root_mean_squared_error: 21.1423
Epoch 3/300
112/112 - 0s - loss: 403.0990 - root_mean_squared_error: 20.0773 - val_loss: 372.8125 - val_root_mean_squared_error: 19.3084
Epoch 4/300
112/112 - 0s - loss: 348.7017 - root_mean_squared_error: 18.6736 - val_loss: 336.3594 - val_root_mean_squared_error: 18.3401
Epoch 5/300
112/112 - 0s - loss: 305.3840 - root_mean_squared_error: 17.4752 - val_loss: 300.4513 - val_root_mean_squared_error: 17.3335
Epoch 6/300
112/112 - 0s - loss: 278.8156 - root_mean_squared_error: 16.6978 - val_loss: 276.4783 - val_root_mean_squared_error: 16.6276
Epoch 7/300
112/112 - 0s - loss: 258.4633 - root_mean_squared_error: 16.0768 - val_loss: 262.6923 - val_root_mean_squared_error: 16.2078
Epoch 8/300
112/112 - 0s - loss: 242.362

In [6]:
ypred = model.predict(X_test)
mean_squared_error(y_test, ypred, squared=False)

7.282532243994006

In [7]:
r2_score(y_test, ypred)

0.922492168769403

In [9]:
!pip install eli5
import eli5
from eli5.sklearn import PermutationImportance

     |████████████████████████████████| 106 kB 7.7 MB/s 


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [10]:
feature_names = [i for i in data.columns if X[i].dtype in [np.int64]]
feature_names

['Cu_promoter',
 'Mn',
 'Ir',
 'Y_promoter',
 'Incipient to wetness impregnation',
 'Wet impregnation',
 'Co-impregnation',
 'Co-precipitation',
 'Sol-gel',
 'Sequential impregnation',
 'Mesoporous Al2O3 (MAl)',
 'BaTiO3',
 'Nanocrystalline MgO',
 'MgAl2O4(nanocry)',
 'Si3N4',
 'MCM-41',
 'ZSM-5',
 'H-ZSM-5',
 'reac He %vol']

In [17]:
from IPython.display import display
perm = PermutationImportance(model,scoring="r2" ,random_state=1).fit(X_val, y_val)
display(eli5.show_weights(perm, feature_names = X_val.columns.tolist()))

Weight,Feature
1.1748 ± 0.1307,react Temperature (C)
0.4426 ± 0.0725,GHSV (mgmin/mL)
0.4028 ± 0.0675,ZrO2
0.3694 ± 0.0744,Wet impregnation
0.3603 ± 0.0467,Incipient to wetness impregnation
0.3584 ± 0.1173,γ-Al2O3(Gamma)
0.2522 ± 0.0454,Ni
0.2513 ± 0.1010,red Temperature (oC)
0.1646 ± 0.0170,Co
0.1349 ± 0.0546,feed_ratio
